In [8]:
import cv2
import numpy as np
import math
import datetime
import ephem


# Load the image
image_path = 'Adogrulama2.jpg'
image = cv2.imread(image_path)

# Show the image and manually select plant and shadow lengths
# First click for the start of the plant, second click for the end of the plant
# Third click for the start of the shadow, fourth click for the end of the shadow

points = []

def click_event(event, x, y, flags, params):
    if event == cv2.EVENT_LBUTTONDOWN:
        points.append((x, y))
        cv2.circle(image, (x, y), 5, (0, 255, 0), -1)
        if len(points) == 4:
            cv2.line(image, points[0], points[1], (255, 0, 0), 2)
            cv2.line(image, points[2], points[3], (0, 0, 255), 2)
        cv2.imshow('image', image)

cv2.imshow('image', image)
cv2.setMouseCallback('image', click_event)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Calculate plant and shadow lengths
def calculate_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

if len(points) == 4:
    plant_length_pixels = calculate_distance(points[0], points[1])
    shadow_length_pixels = calculate_distance(points[2], points[3])
    
    # Define scale factor for real-world dimensions
    scale_factor = 0.0026458333   # centimeters per pixel
    plant_length_real = plant_length_pixels * scale_factor
    shadow_length_real = shadow_length_pixels * scale_factor
    
    print(f"Plant Length (pixels): {plant_length_pixels}")
    print(f"Shadow Length (pixels): {shadow_length_pixels}")
    print(f"Plant Length (real): {plant_length_real} m")
    print(f"Shadow Length (real): {shadow_length_real} m")

    # Calculate solar position (azimuth and altitude)
    # Assume default observer coordinates for Turkey (latitude: 39.9334, longitude: 32.8597)
    observer_latitude = math.radians(39.9334)  # Observer's latitude (in radians)
    observer_longitude = math.radians(32.8597)  # Observer's longitude (in radians)

    # Calculate solar position based on azimuth and altitude
    def calculate_solar_position(azimuth_deg, altitude_deg, observer_latitude, observer_longitude):
        # Convert azimuth to radians
        azimuth_rad = math.radians(azimuth_deg)
        
        # Calculate solar declination angle
        sin_declination = math.sin(math.radians(altitude_deg)) * math.sin(observer_latitude) + \
                          math.cos(math.radians(altitude_deg)) * math.cos(observer_latitude) * math.cos(azimuth_rad)
        solar_declination = math.asin(sin_declination)
        
        # Calculate solar hour angle
        cos_hour_angle = (math.sin(math.radians(altitude_deg)) - math.sin(observer_latitude) * math.sin(solar_declination)) / \
                         (math.cos(observer_latitude) * math.cos(solar_declination))
        solar_hour_angle = math.acos(cos_hour_angle)
        
        # Calculate local solar hour angle
        if math.sin(azimuth_rad) > 0:
            solar_hour_angle = solar_hour_angle
        else:
            solar_hour_angle = -solar_hour_angle
        
        # Convert solar declination and solar hour angle to degrees
        solar_declination_deg = math.degrees(solar_declination)
        solar_hour_angle_deg = math.degrees(solar_hour_angle)
        
        return solar_declination_deg, solar_hour_angle_deg
    
    # For this example, assume default first two click points (start and end of plant)
    # You would use the real angles calculated directly from these points after determining them.
    # For example:
    azimuth_deg = 45  # Sample azimuth angle (degrees)
    altitude_deg = 30  # Sample altitude angle (degrees)
    
    solar_declination_deg, solar_hour_angle_deg = calculate_solar_position(azimuth_deg, altitude_deg, observer_latitude, observer_longitude)
    
    print(f"Solar Declination Angle: {solar_declination_deg} degrees")
    print(f"Solar Hour Angle: {solar_hour_angle_deg} degrees")

else:
    print("Please click four points to select plant and shadow lengths.")




    


Plant Length (pixels): 112.21853679316978
Shadow Length (pixels): 98.03060746521976
Plant Length (real): 0.2969115415246438 m
Shadow Length (real): 0.259372645650707 m
Solar Declination Angle: 52.23320645949901 degrees
Solar Hour Angle: 90.90602437061628 degrees


In [7]:
# Azimuth and elevation anlges of sun (degree units)
sun_azimuth = 209.51  
sun_elevation = 59.80  

# Date time information
date = '2021/08/19 13:51:00'  # Örnek tarih ve saat, UTC cinsinden

# Creating ephem observation
observer = ephem.Observer()
observer.date = date

# Calculating observer
sun = ephem.Sun()
sun.compute(observer)

# Fence the right ascension and declination of sun
sun_ra = sun.ra  
sun_dec = 52.23320645949901 

# Convert to radian
azimuth_rad = math.radians(sun_azimuth)
elevation_rad = math.radians(sun_elevation)

# Calculating lat and long of observer 
observer_lat = math.asin(math.sin(elevation_rad) * math.sin(sun_dec) + math.cos(elevation_rad) * math.cos(sun_dec) * math.cos(azimuth_rad))
observer_lon = sun_ra - math.atan2(-math.sin(azimuth_rad), math.cos(elevation_rad) * math.tan(sun_dec) - math.sin(elevation_rad) * math.cos(azimuth_rad))

# Convert to degree 
observer_lat_deg = math.degrees(observer_lat)
observer_lon_deg = math.degrees(observer_lon)


print(f"Enlem (Latitude): {observer_lat_deg:.6f}°")
print(f"Boylam (Longitude): {observer_lon_deg:.6f}°")

Enlem (Latitude): 75.086053°
Boylam (Longitude): 16.742227°
